# Getting Started with Pilot-Streaming and Quantum Computing (on local environment or remote VM via ssh)

In the first step we need to import all required packages and modules into the Python Path

Pilot-Streaming can be used to manage the Dask and Kafka environments both in the cloud and on the edge. 



`resource`: URL of the Local Resource Manager. Examples:

* `slurm://localhost`: Submit to local SLURM resource manager, e.g. on master node of Wrangler or Stampede
* `slurm+ssh://login1.wrangler.tacc.utexas.edu`: Submit to Wrangler master node SLURM via SSH (e.g. on node running a job)
* `os://` Openstack
* `ec2://` EC2


`type:` The `type` attributes specifies the cluster environment. It can be: `Spark`, `Dask` or `Kafka`.

***For standard task-level parallelism `Dask` is recommended.***


Depending on the resource there might be other configurations necessary, e.g. to ensure that the correct subnet is used the Spark driver can be configured using various environment variables:   os.environ["SPARK_LOCAL_IP"]='129.114.58.2'



In [ ]:
# Pilot-Streaming
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import distributed
import json
import pilot.streaming
import getpass
import socket
import logging
sys.modules['pilot.streaming']

#RESOURCE_URL_HPC="slurm+ssh://login4.stampede2.tacc.utexas.edu"
RESOURCE_URL_HPC="ssh://localhost"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")


# 1. Dask on local machine

In [ ]:
pilot_compute_description_dask = {
    "resource":"ssh://{}@localhost".format(getpass.getuser()),
    "working_directory": os.path.join(os.path.expanduser("~"), "work"),
    "number_cores": 48,
    "queue": "normal",
    "walltime": 59,
    "type":"dask"
}

In [ ]:
##%%time
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description_dask)
dask_pilot.wait()
dask_pilot.get_details()

## Test edge dask behind firewall

In [ ]:
dask_client  = distributed.Client(dask_pilot.get_details()['master_url'])
#dask_client  = distributed.Client()
dask_client.scheduler_info()

In [ ]:
dask_client.gather(dask_client.map(lambda a: a*a, range(10)))

In [ ]:
dask_client.gather(dask_client.map(lambda a: socket.gethostname(), range(10)))

# Create simple quantum circuit for simulation

Requirements:

        pip install pennylane 

In [ ]:
import pennylane as qml
from timeit import default_timer as timer


def run_circuit():
    wires = 4
    layers = 1
    num_runs = 50
    GPUs = 1

    dev = qml.device('default.qubit', wires=wires, shots=None)

    @qml.qnode(dev)
    def circuit(parameters):
        qml.StronglyEntanglingLayers(weights=parameters, wires=range(wires))
        return [qml.expval(qml.PauliZ(i)) for i in range(wires)]

    shape = qml.StronglyEntanglingLayers.shape(n_layers=layers, n_wires=wires)
    weights = qml.numpy.random.random(size=shape)
    val = circuit(weights)
    return val

In [ ]:
dask_client.gather(dask_client.map(lambda a: run_circuit(), range(10)))